In [78]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
import joblib
import matplotlib.pyplot as plt
import seaborn as sns



In [79]:
train_data = pd.read_excel('train_data.xlsx')

In [80]:
# Handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [81]:
# 1. Address missing values
train_data.fillna(train_data.median(numeric_only=True), inplace=True)
train_data.fillna("Unknown", inplace=True)

In [82]:
# 2. Encode categorical variables
categorical_columns = ['purpose', 'home_ownership', 'verification_status', 'application_type']
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    label_encoders[col] = le


In [83]:

# 3. Normalize numeric variables
scaler = StandardScaler()
numeric_columns = ['annual_inc', 'loan_amnt', 'account_bal', 'installment', 'cibil_score']
train_data[numeric_columns] = scaler.fit_transform(train_data[numeric_columns])


In [84]:
# 4. Create derived features
train_data['debt_to_income_ratio'] = train_data['installment'] / (train_data['annual_inc'] + 1e-6)


In [85]:
# 5. Prepare data for modeling
X = train_data.drop(['loan_status', 'customer_id', 'transaction_date','term','sub_grade','home_ownership','total_no_of_acc','purpose','application_type','verification_status','installment',"account_bal"], axis=1)
y = train_data['loan_status']

In [86]:
X

cibil_score  annual_inc  int_rate  loan_amnt  emp_length  \
0         -0.848174   -0.022944     16.24  -0.875364          11   
1         -1.066790   -0.105580     23.40  -1.007359          11   
2         -1.066790    0.026638     17.57  -0.787367          11   
3          0.244907    0.770368     12.85  -0.567375           2   
4         -0.848174   -0.138635     14.98  -0.567375          11   
...             ...         ...       ...        ...         ...   
113700    -0.629558   -0.303908     16.29  -0.567375          11   
113701    -1.066790   -0.518763     17.27  -0.894613           2   
113702     0.244907   -0.766673     16.29  -0.567375           3   
113703    -0.192326    0.142330     15.31  -0.278636           7   
113704    -0.848174   -0.386544     19.99  -0.347383           1   

        debt_to_income_ratio  
0                  32.538176  
1                   7.793372  
2                 -23.333370  
3                  -0.571111  
4                   5.820099  
...                      ...  
113700              1.241747  
113701              1.457558  
113702              0.492226  
113703             -0.384260  
113704              1.315272  

[113705 rows x 6 columns]

In [87]:
# Handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [88]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [89]:
# 6. Train a Random Forest model
rf_model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10, class_weight='balanced')
rf_model.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', max_depth=10, random_state=42)

In [90]:
# 7. Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [91]:
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.73

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.71      0.72     16996
           1       0.71      0.75      0.73     16611

    accuracy                           0.73     33607
   macro avg       0.73      0.73      0.73     33607
weighted avg       0.73      0.73      0.73     33607


Confusion Matrix:
 [[11987  5009]
 [ 4154 12457]]


In [92]:
from sklearn.linear_model import LogisticRegression
rf_model = LogisticRegression()
rf_model.fit(X_train, y_train)



LogisticRegression()

In [93]:
# 7. Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [94]:
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.65

Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.60      0.63     16996
           1       0.63      0.70      0.66     16611

    accuracy                           0.65     33607
   macro avg       0.65      0.65      0.65     33607
weighted avg       0.65      0.65      0.65     33607


Confusion Matrix:
 [[10162  6834]
 [ 5003 11608]]


**bold text**

### Chosen Model: RANDOM FOREST

Model performance for Training set
- Accuracy: 0.73
- F1 score: 0.73
- Precision: 0.71
- Recall: 0.75

----------------------------------
Model performance for Test set
- Accuracy: 0.66
- F1 score: 0.76
- Precision: 0.70
- Recall: 0.84
- support:5400

The Random Forest model shows a solid balance between accuracy, precision, recall, F1 score on both the training and test sets.
Its performance on the test set (accuracy: 0.66, F1 score: 0.76, Precision: 0.70) making it a reliable choice for the task.